In [1]:
# Get preprocessing methods from make_network.ipynb to keep consistent.
RANDOM_STATE = 5664
import os
import pandas as pd
def load_and_clean_data(filepath):
    """
    Load Reddit data and remove duplicate user-subreddit combinations.
    
    Args:
        filepath (str): Path to the CSV data file
        
    Returns:
        pandas.DataFrame: Cleaned dataframe with unique user-subreddit combinations
    """
    try:
        # Check if file exists
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Data file not found: {filepath}")
            
        # Load the dataset
        print(f"Loading data from {filepath}...")
        df = pd.read_csv(filepath)
        print(f"Original data shape: {df.shape}")
        
        # Remove duplicate user-subreddit combinations
        df_unique = df.drop_duplicates(subset=['author_name', 'subreddit']).copy()
        
        print(f"Data shape after cleaning: {df_unique.shape}")
        
        return df_unique
    
    except Exception as e:
        print(f"Error in load_and_clean_data: {str(e)}")
        raise
    
def analyze_post_dates(df):
    # Convert post_created_time to datetime
    df['post_created_time'] = pd.to_datetime(df['post_created_time'])
    
    # Get the earliest and latest dates
    min_date = df['post_created_time'].min()
    max_date = df['post_created_time'].max()
    
    return min_date, max_date

def filter_by_date(df, start_date=None, end_date=None):
    """
    Filter dataframe to include only posts within a specific date range.
    
    Args:
        df (pandas.DataFrame): DataFrame with 'post_created_time' column
        start_date (str, datetime, optional): Keep posts on or after this date
        end_date (str, datetime, optional): Keep posts on or before this date
        
    Returns:
        pandas.DataFrame: Filtered dataframe
    """
    # Make sure post_created_time is datetime
    if not pd.api.types.is_datetime64_dtype(df['post_created_time']):
        df['post_created_time'] = pd.to_datetime(df['post_created_time'])
    
    original_count = len(df)
    
    # Apply date filters
    if start_date is not None:
        if isinstance(start_date, str):
            start_date = pd.to_datetime(start_date)
        df = df[df['post_created_time'] >= start_date]
    
    if end_date is not None:
        if isinstance(end_date, str):
            end_date = pd.to_datetime(end_date)
        df = df[df['post_created_time'] <= end_date]
    
    # Report on filtering
    print(f"Date filtering:")
    if start_date is not None:
        print(f"  Start date: {start_date}")
    if end_date is not None:
        print(f"  End date: {end_date}")
    print(f"  Original records: {original_count}")
    print(f"  Filtered records: {len(df)} ({len(df)/original_count*100:.1f}% retained)")
    
    return df

## Load data

In [2]:
from datetime import timedelta

df_clean = load_and_clean_data("reddit_opinion_ru_ua.csv")
min_date, max_date = analyze_post_dates(df_clean)
cutoff_date = max_date - timedelta(days=10)

df_recent = filter_by_date(df_clean, start_date=cutoff_date)

Loading data from reddit_opinion_ru_ua.csv...
Original data shape: (5168018, 24)
Data shape after cleaning: (766897, 24)
Date filtering:
  Start date: 2025-04-19 11:00:47
  Original records: 766897
  Filtered records: 41732 (5.4% retained)


In [3]:
todrop = ["comment_id", "created_time","post_id","user_is_verified","user_account_created_time", "user_awardee_karma", "user_awarder_karma", "user_comment_karma", "user_link_karma", "post_created_time"]
df_recent.drop(todrop, axis=1, inplace=True)
df_recent

C:\Users\brand\AppData\Local\Temp\ipykernel_16632\2139921290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recent.drop(todrop, axis=1, inplace=True)


,score,self_text,subreddit,author_name,controversiality,ups,downs,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received
0,1,I'd have to agree that it's hard to shop for m...,AskARussian,rsaachit,0,1,0,2272.0,8,hello!! I’m currently trying to come up with g...,Gifts for Russian man - from an American,0.78,8,0
1,1,"They don't, so you don't have to worry about t...",europe,potatolulz,0,1,0,605528.0,590,NaN,Zelensky dismisses Putin’s declaration of a 72...,0.98,590,0
2,1,Trump is just buying time. He won't do shit t...,UkraineWarVideoReport,Many-Cartographer-45,0,1,0,2492.0,134,NaN,The real Putin is now clear to Trump - and his...,0.95,134,0
3,1,They are being randomly called up for military...,worldnews,Corka,0,1,0,137057.0,2185,NaN,"Russia has lost over 950,000 soldiers since Fe...",0.97,2185,0
4,1,Most of your assumptions here are wrong.\n\nFi...,worldnews,LeSygneNoir,0,1,0,212668.0,2185,NaN,"Russia has lost over 950,000 soldiers since Fe...",0.97,2185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98869,72,Someone has been playing Metro…,CombatFootage,GallahadTheGreat,0,72,0,11439.0,4279,NaN,The most unusual piece of equipment used by th...,0.99,4279,0
98884,4,Time for Europe to take more responsibility.,UkraineWarVideoReport,Still_Internet9670,0,4,0,2747.0,233,NaN,Trump administration not discussing new aid pa...,0.95,233,0
98991,425,Whoever mixed the music in this video clearly ...,CombatFootage,helgur,0,425,0,69011.0,4279,NaN,The most unusual piece of equipment used by th...,0.99,4279,0
99003,850,Russia: I choo-choo-choose violence.,CombatFootage,DasFunktopus,0,850,0,48910.0,4279,NaN,The most unusual piece of equipment used by th...,0.99,4279,0


In [4]:
# Get basic information
print("Number of posts:", len(df_recent))
print("All subreddits:")
pd.DataFrame(df_recent.subreddit.explode().unique())

Number of posts: 41732
All subreddits:


,0
0,AskARussian
1,europe
2,UkraineWarVideoReport
3,worldnews
4,UkraineRussiaReport
5,conspiracy
6,ukraine
7,ANormalDayInRussia
8,UkrainianConflict
9,russiawarinukraine


## Set up for topic modeling and sentiment analysis for comments, posts, and post titles

In [5]:
# nltk.download("stopwords")
# nltk.download("punkt_tab")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import *

def preprocess_one_doc(text: str, lower: bool, punct: bool, stop: bool, stem: bool):
    puncts = set(string.punctuation)
    puncts.add("...") # punct does not contain ellipses
    puncts.add("…")
    puncts.add("===")
    puncts.add("—")
    puncts.add("–")
    puncts.add("“")
    puncts.add("”")
    puncts.add("’")
    puncts.add("‘")
    stops = set(stopwords.words("english"))
    # stops.add("")
    
    
    # Lowercase the words depending on sentiment or topic modeling
    if lower:
        step0 = text.lower()
    else:
        step0 = text
    step1 = word_tokenize(step0)
    
    
    if punct:
        step2 = "".join([ch for ch in " ".join(step1) if ch not in puncts]).split()
    else:
        step2 = step1
        
        
    
    if stop:
        # Remove stopwords
        step3 = [token for token in step2
                    if token not in stops # drop stopwords
                    # and len(token) > 2 # drop words of insignificant length
                    and (not token.startswith("http"))] # drop links
    else:
        step3 = step2
        
    if stem:
        stemmer = PorterStemmer()
        step4 = [stemmer.stem(i) for i in step3]
    else:
        step4 = step3
        
    return step4

from gensim import corpora, models

def make_dictionary(alltexts):
    d = corpora.Dictionary(alltexts)
    d.filter_extremes(no_below=5, no_above=0.3) # Keep words that are in more than 5 documents, but not in more than a third of all documents
    d.compactify()
    return d

from langdetect import detect, DetectorFactory
DetectorFactory.seed = RANDOM_STATE

def filter_english(text):
    try:
        return detect(text) == "en"
    except:
        return False


def make_all_components(df, text_column_name):
    # Drop all missing values
    dfc = df.copy()
    dfc.dropna(subset=[text_column_name], inplace=True)
    
    # Filter non-english text
    is_english = dfc[text_column_name].apply(filter_english)
    dfc = dfc[is_english]
    
    # Create with standard preprocessing
    preprocessed = dfc[text_column_name].apply(lambda x: preprocess_one_doc(x, lower=True, stop=True, punct=True, stem=True)) # Preprocess all docs
    dictionary = make_dictionary(preprocessed.tolist()) # Use list of lists of strings
    corpus = [dictionary.doc2bow(text) for text in preprocessed.tolist()] # bag of words representation of documents
    return preprocessed, dictionary, corpus

In [6]:
# Create with standard preprocessing
preprocessed_comments, dictionary_comments, corpus_comments = make_all_components(df_recent, "self_text")
preprocessed_post_content, dictionary_post_content, corpus_post_content = make_all_components(df_recent, "post_self_text")
preprocessed_title, dictionary_title, corpus_title = make_all_components(df_recent, "post_title")

In [7]:
preprocessed_comments

0        [agre, hard, shop, men, unless, nich, hobbi, l...
1                                          [nt, nt, worri]
2        [trump, buy, time, wo, nt, shit, help, defeat,...
3        [randomli, call, militari, servic, period, one...
4        [assumpt, wrong, first, consid, overal, size, ...
                               ...                        
98869                                [someon, play, metro]
98884                         [time, europ, take, respons]
98991    [whoever, mix, music, video, clearli, miss, go...
99003                     [russia, choochoochoos, violenc]
99048    [2025, got, russian, hors, logist, convoy, lar...
Name: self_text, Length: 38171, dtype: object

## Evaluate to find best number of topics

In [8]:
# Taken from https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920

from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

def eval_model(model, text, dic, corp):
    # Compute Perplexity
    perp = model.log_perplexity(corp)
    # a measure of how good the model is. lower the better.
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=model, texts=text, dictionary=dic, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    
    return perp, coherence

def plot_evals(perps, coherences, ks):
    
    fig = plt.figure("Perplexity and Coherence Analysis", figsize=(8, 8))
    axgrid = fig.add_gridspec(1, 2)
    
    ax1 = fig.add_subplot(axgrid[0, 0])
    ax1.plot(ks, perps)
    ax1.set_title("Number of topics vs Perplexity Score")
    ax1.set_xlabel("Number of Topics")
    ax1.set_ylabel("Perplexity Score")
    ax1.grid()
    
    ax2 = fig.add_subplot(axgrid[0, 1])
    ax2.plot(ks, coherences)
    ax2.set_title("Number of topics vs Coherence Score")
    ax2.set_xlabel("Number of Topics")
    ax2.set_ylabel("Coherence Score")
    ax2.grid()
    
    fig.tight_layout()
    plt.show()
    
    
def full_model_test_loop(text, corpus, dictionary):
    ks = [1,2,3,4,5,6,7,8,9,10, 20, 30]
    perps = []
    coherences = []
    for k in ks:
        ldamodel = models.ldamodel.LdaModel(corpus, num_topics=k, id2word=dictionary, passes=20, random_state=RANDOM_STATE)
        scores = eval_model(ldamodel, text.tolist(), dictionary, corpus)
        perps.append(scores[0])
        coherences.append(scores[1])
    plot_evals(perps, coherences, ks)

In [9]:
# print("Comments eval:")
# full_model_test_loop(preprocessed_comments, corpus_comments, dictionary_comments)
# print("Post Content eval:")
# full_model_test_loop(preprocessed_post_content, corpus_post_content, dictionary_post_content)
# print("Post Title eval:")
# full_model_test_loop(preprocessed_title , corpus_title, dictionary_title)

# 22 minutes to run all

## Topic Modeling

In [10]:
ldamodel_comments = models.ldamodel.LdaModel(corpus_comments, num_topics=10, id2word=dictionary_comments, passes=20, random_state=RANDOM_STATE)
ldamodel_post_content = models.ldamodel.LdaModel(corpus_post_content, num_topics=5, id2word=dictionary_post_content, passes=20, random_state=RANDOM_STATE)
ldamodel_title = models.ldamodel.LdaModel(corpus_title, num_topics=20, id2word=dictionary_title, passes=20, random_state=RANDOM_STATE)

# 6 min 30 sec

In [ ]:
# TODO: call eval_model to get the perplexity and coherence of the top k model and present them in table

In [11]:
print("Comment topics:")
ldamodel_comments.show_topics()

Comment topics:


[(0,
  '0.034*"war" + 0.033*"day" + 0.032*"said" + 0.029*"gt" + 0.021*"end" + 0.020*"bomb" + 0.019*"one" + 0.019*"na" + 0.014*"gon" + 0.014*"moscow"'),
 (1,
  '0.064*"ukrain" + 0.050*"russia" + 0.020*"nt" + 0.020*"war" + 0.018*"want" + 0.018*"would" + 0.015*"peac" + 0.014*"us" + 0.012*"putin" + 0.012*"give"'),
 (2,
  '0.085*"fuck" + 0.033*"easter" + 0.031*"yeah" + 0.026*"amp" + 0.020*"god" + 0.017*"oh" + 0.014*"hour" + 0.013*"hear" + 0.013*"church" + 0.013*"pleas"'),
 (3,
  '0.031*"kill" + 0.026*"thank" + 0.018*"orang" + 0.017*"shit" + 0.014*"peopl" + 0.013*"children" + 0.011*"say" + 0.011*"show" + 0.010*"red" + 0.010*"human"'),
 (4,
  '0.079*"trump" + 0.049*"putin" + 0.028*"like" + 0.020*"look" + 0.012*"make" + 0.010*"presid" + 0.010*"zelenski" + 0.009*"play" + 0.008*"get" + 0.008*"biden"'),
 (5,
  '0.054*"nt" + 0.025*"peopl" + 0.017*"say" + 0.015*"know" + 0.015*"think" + 0.012*"like" + 0.011*"would" + 0.010*"believ" + 0.010*"want" + 0.010*"american"'),
 (6,
  '0.027*"year" + 0.023*"s

In [12]:
print("Post content topics:")
ldamodel_post_content.show_topics()

Post content topics:


[(0,
  '0.017*"war" + 0.012*"trump" + 0.012*"video" + 0.011*"sourc" + 0.010*"ukrain" + 0.010*"us" + 0.009*"inform" + 0.008*"realiti" + 0.008*"document" + 0.008*"educ"'),
 (1,
  '0.012*"would" + 0.011*"ukrain" + 0.011*"want" + 0.008*"like" + 0.008*"amp" + 0.007*"live" + 0.007*"peopl" + 0.007*"get" + 0.007*"ukrainian" + 0.006*"realli"'),
 (2,
  '0.011*"would" + 0.009*"like" + 0.009*"nt" + 0.009*"lawn" + 0.007*"name" + 0.006*"peopl" + 0.006*"think" + 0.005*"said" + 0.005*"elect" + 0.005*"time"'),
 (3,
  '0.018*"soldier" + 0.010*"ukrainian" + 0.010*"ukrain" + 0.008*"colombian" + 0.008*"one" + 0.006*"forc" + 0.006*"unit" + 0.006*"said" + 0.006*"gt" + 0.005*"drone"'),
 (4,
  '0.013*"banksi" + 0.011*"nt" + 0.008*"artist" + 0.007*"would" + 0.007*"administr" + 0.006*"rob" + 0.006*"trump" + 0.006*"never" + 0.005*"work" + 0.005*"time"')]

In [13]:
print("Title topics:")
ldamodel_title.show_topics()

Title topics:


[(2,
  '0.133*"russia" + 0.082*"hour" + 0.078*"fight" + 0.065*"offic" + 0.062*"time" + 0.050*"violat" + 0.043*"first" + 0.042*"offici" + 0.034*"admit" + 0.032*"troop"'),
 (4,
  '0.225*"truce" + 0.190*"ceasefir" + 0.078*"ua" + 0.074*"sinc" + 0.074*"pov" + 0.047*"may" + 0.041*"announc" + 0.026*"territori" + 0.022*"includ" + 0.018*"order"'),
 (19,
  '0.119*"claim" + 0.078*"air" + 0.060*"place" + 0.058*"back" + 0.056*"play" + 0.048*"within" + 0.048*"gener" + 0.046*"jet" + 0.043*"await" + 0.040*"true"'),
 (15,
  '0.105*"attack" + 0.093*"moscow" + 0.080*"along" + 0.072*"join" + 0.065*"show" + 0.049*"pope" + 0.049*"nsfw" + 0.046*"negoti" + 0.039*"may" + 0.034*"9"'),
 (11,
  '0.099*"trump" + 0.081*"peopl" + 0.077*"russia" + 0.076*"plan" + 0.074*"propos" + 0.051*"away" + 0.046*"reject" + 0.045*"one" + 0.044*"launch" + 0.043*"warn"'),
 (18,
  '0.245*"putin" + 0.128*"war" + 0.126*"trump" + 0.116*"declar" + 0.061*"end" + 0.048*"call" + 0.045*"start" + 0.033*"donald" + 0.032*"say" + 0.026*"deal"'),

## Sentiment Analysis

In [15]:
#TODO adapt to generic scores, include average pos, neg, neu scores, plot histogram of scores, plot distribution of compound scores

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
sid = SentimentIntensityAnalyzer()

def find_compound_sentiment(document):
    sentences = sent_tokenize(document)
    s = 0
    for sentence in sentences:
        sentiment = sid.polarity_scores(sentence)["compound"]
        s += sentiment
    return s/len(sentences)
    
def find_all_topic_sentiments(corp, raws, model, refs):
    dominant_topics = []
    document_scores = []
    corp_plus_raws = zip(corp, raws)
    for bag, raw in corp_plus_raws:
        topics = model.get_document_topics(bag)
        dominant_topic = sorted(topics, key=lambda x: -x[1])[0][0]
        dominant_topics.append(dominant_topic)
        document_scores.append(find_compound_sentiment(raw))
        
    topic_sentiments = {}
    topic_refs = {}
    document_topic_score = list(zip(dominant_topics, document_scores, refs))
    for i in range(k):
        sentiments_in_topic = [sentiment for topic, sentiment, _ in document_topic_score if topic == i]
        topic_sentiments[i] = sum(sentiments_in_topic)/len(sentiments_in_topic)
        refs_in_topic = [refs for topic, _, refs in document_topic_score if topic == i]
        topic_refs[i] = sum(refs_in_topic)/len(refs_in_topic)
        
    return [topic_sentiments, document_scores, topic_refs]

In [17]:
topic_scores, dataset_scores, topic_nrefs = find_all_topic_sentiments(corpus, dataset["content"], ldamodel, dataset["nrefs"])

NameError: name 'corpus' is not defined